In [ ]:
#imports
import numpy as np
import pandas as pd

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
#dados
train = pd.read_csv('/content/drive/MyDrive/titanic/train.csv')
test = pd.read_csv('/content/drive/MyDrive/titanic/test.csv')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
print(train.shape)
print(test.shape)

(891, 12)
(418, 11)


In [ ]:
train.describe()

PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200

In [ ]:
#tratando os dados
#separando por titulo
for i in [train,test]:
    i["Title"] = i["Name"].apply(lambda x: x.split(',')[1].split('.')[0].strip() if ',' in x and '.' in x.split(',')[1] else '')
    print(i["Title"].unique())
    i["Title"] = i["Title"].replace([" Don", " Rev", " Dr", " Major", " Lady", " Sir", " Col", " Capt", " the Countess", " Jonkheer"], "Rare")
    i["Title"] = i["Title"].replace([" Mlle", " Ms"], " Miss")
    i["Title"] = i["Title"].replace([" Mme", " Mrs"], " Mr")

['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'the Countess' 'Jonkheer']
['Mr' 'Mrs' 'Miss' 'Master' 'Ms' 'Col' 'Rev' 'Dr' 'Dona']


In [ ]:
target='Survived'
x_train,x_test,y_train,y_test=train_test_split(train.drop(target,axis=1),train[target],test_size=.3,random_state=0)
df_train=pd.concat([x_train,y_train],axis=1)
df_test=pd.concat([x_test,y_test],axis=1)
for i in [df_train,df_test,test]:
    i.drop(['Name','Ticket','PassengerId'],axis=1,inplace=True)

In [ ]:
print('train no. of unique values')
print(df_train.nunique())
print('test no. of unique values')
print(df_test.nunique())

train no. of unique values
Pclass        3
Sex           2
Age          83
SibSp         7
Parch         7
Fare        216
Cabin       120
Embarked      3
Title        14
Survived      2
dtype: int64
test no. of unique values
Pclass        3
Sex           2
Age          64
SibSp         6
Parch         5
Fare        134
Cabin        48
Embarked      3
Title         9
Survived      2
dtype: int64


In [ ]:
categorical_features=[i for i in df_train.columns if df_train[i].dtype=='O']
numerical_features=[i for i in df_train.columns if df_train[i].dtype!='O']
print('categorical_features :\n',categorical_features,'\nnumerical_features :\n',numerical_features)

categorical_features :
 ['Sex', 'Cabin', 'Embarked', 'Title'] 
numerical_features :
 ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']


In [ ]:
#irq removendo outliers
for feature in numerical_features:
    if 0 in df_train[feature].unique():
        pass
    else:
        q1=df_train[feature].quantile(0.25)
        q3=df_train[feature].quantile(0.75)
        iqr=q3-q1
        lower_limit=q1-1.5*iqr
        upper_limit=q3+1.5*iqr
        df_train=df_train[(df_train[feature]<upper_limit) & (df_train[feature]>lower_limit)]


In [ ]:
#tratando dados nulos
#preenchendo valores ausentes em 'Age' com a mediana
df_train['Age'].fillna(df_train['Age'].median(), inplace=True)
df_test['Age'].fillna(df_test['Age'].median(), inplace=True)
test['Age'].fillna(test['Age'].median(), inplace=True)

#preenchendo valores ausentes em 'Embarked' com o valor mais frequente
df_train['Embarked'].fillna(df_train['Embarked'].mode()[0], inplace=True)
df_test['Embarked'].fillna(df_test['Embarked'].mode()[0], inplace=True)
test['Embarked'].fillna(test['Embarked'].mode()[0], inplace=True)

#preenchendo valores ausentes em 'Fare' com a mediana
df_test['Fare'].fillna(df_test['Fare'].median(), inplace=True)
test['Fare'].fillna(test['Fare'].median(), inplace=True)

print(df_train.isnull().sum())
print(df_test.isnull().sum())
print(test.isnull().sum())

Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Cabin       359
Embarked      0
Title         0
Survived      0
dtype: int64
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Cabin       216
Embarked      0
Title         0
Survived      0
dtype: int64
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Cabin       327
Embarked      0
Title         0
dtype: int64


In [ ]:
print(train.shape)
print(test.shape)

(891, 13)
(418, 9)


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in categorical_features:
  df_train[i] = le.fit_transform(df_train[i])
  df_test[i] = le.fit_transform(df_test[i])
  test[i] = le.fit_transform(test[i])


In [ ]:
X_train = df_train.drop('Survived', axis=1)
y_train = df_train['Survived']
X_test = df_test.drop('Survived', axis=1)
y_test = df_test['Survived']

In [ ]:
model = LogisticRegression()


In [ ]:
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
#metricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn+fp)

print("Acurácia do Treino:", accuracy)
print("Precisão:", precision)
print("Especificidade:", specificity)
print("Recall:", recall)
print("F1 Score:", f1)

Acurácia do Treino: 0.7947761194029851
Precisão: 0.7272727272727273
Especificidade: 0.8392857142857143
Recall: 0.72
F1 Score: 0.7236180904522613


In [ ]:
print(test.columns)

if 'PassengerId' not in test.columns:
    test['PassengerId'] = range(892, 892 + len(test))

results = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': model.predict(test.drop('PassengerId', axis=1))
})


results.to_csv('titanicRL.csv', index=False)

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked',
       'Title'],
      dtype='object')
